In [2]:
import sqlite3

conn = sqlite3.connect("kbo_database.db")
cursor = conn.cursor()

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('denomination',), ('branch',), ('activity',), ('establishment',), ('address',), ('contact',), ('meta',), ('code',), ('enterprise',), ('main1',), ('main',)]


In [7]:
cursor.executescript("""
DROP TABLE IF EXISTS main;

CREATE TABLE main AS
SELECT 
	enterprise.EnterpriseNumber,
	denom.Denomination AS company_name, 
	SUBSTR(CAST(clss.NaceCode AS TEXT), 1, 1) AS sector,
	CASE
		WHEN adr.Zipcode BETWEEN 1000 AND 1299 THEN 'Brussels'
		WHEN adr.Zipcode BETWEEN 1300 AND 1499 THEN 'Walloon Brabant'
		WHEN adr.Zipcode BETWEEN 1500 AND 1999 THEN 'Flemish Brabant'
		WHEN adr.Zipcode BETWEEN 2000 AND 2999 THEN 'Antwerp'
		WHEN adr.Zipcode BETWEEN 3000 AND 3999 THEN 'Flemish Brabant'
		WHEN adr.Zipcode BETWEEN 4000 AND 4999 THEN 'Liège'
		WHEN adr.Zipcode BETWEEN 5000 AND 5999 THEN 'Namur'
		WHEN adr.Zipcode BETWEEN 6000 AND 6999 THEN 'Hainaut'
		WHEN adr.Zipcode BETWEEN 7000 AND 7999 THEN 'Hainaut'
		WHEN adr.Zipcode BETWEEN 8000 AND 8999 THEN 'West Flanders'
		WHEN adr.Zipcode BETWEEN 9000 AND 9999 THEN 'East Flanders'
		ELSE 'Unknown'
    	END AS province,
	adr.Zipcode,
	clss.Classification,
	enterprise.StartDate,
	clss.ActivityGroup AS private_public
	
FROM enterprise
	JOIN 
		(SELECT Classification, NaceCode, EntityNumber, ActivityGroup FROM activity WHERE Classification = 'MAIN') AS clss ON clss.EntityNumber = enterprise.EnterpriseNumber
	LEFT JOIN 
		(SELECT Zipcode, EntityNumber FROM address) AS adr ON adr.EntityNumber = enterprise.EnterpriseNumber
  	JOIN 
		(SELECT Code, Description, Category FROM code WHERE Language = 'FR' AND Category = 'ActivityGroup') AS cd ON cd.Code = clss.ActivityGroup
	JOIN 
		(SELECT Denomination, EntityNumber FROM denomination WHERE TypeOfDenomination = 1) AS denom ON denom.EntityNumber = enterprise.EnterpriseNumber
	

;

""")

In [ ]:
"""SELECT 
  enterprise.EnterpriseNumber, 
  adr.Zipcode 
FROM enterprise
LEFT JOIN address AS adr ON adr.EntityNumber = enterprise.EnterpriseNumber
LIMIT 20;"""

In [ ]:
cursor.execute("PRAGMA table_info(enterprise);")
enterprise_info = cursor.fetchall()
print("Structure de 'enterprise' :")
for col in enterprise_info:
    print(col)


cursor.execute("PRAGMA table_info(address);")
address_info = cursor.fetchall()
print("\nStructure de 'address' :")
for col in address_info:
    print(col)



Structure de 'enterprise' :
(0, 'EnterpriseNumber', 'TEXT', 0, None, 0)
(1, 'Status', 'TEXT', 0, None, 0)
(2, 'JuridicalSituation', 'INTEGER', 0, None, 0)
(3, 'TypeOfEnterprise', 'INTEGER', 0, None, 0)
(4, 'JuridicalForm', 'REAL', 0, None, 0)
(5, 'JuridicalFormCAC', 'REAL', 0, None, 0)
(6, 'StartDate', 'TEXT', 0, None, 0)

Structure de 'address' :
(0, 'EntityNumber', 'TEXT', 0, None, 0)
(1, 'TypeOfAddress', 'TEXT', 0, None, 0)
(2, 'CountryNL', 'TEXT', 0, None, 0)
(3, 'CountryFR', 'TEXT', 0, None, 0)
(4, 'Zipcode', 'TEXT', 0, None, 0)
(5, 'MunicipalityNL', 'TEXT', 0, None, 0)
(6, 'MunicipalityFR', 'TEXT', 0, None, 0)
(7, 'StreetNL', 'TEXT', 0, None, 0)
(8, 'StreetFR', 'TEXT', 0, None, 0)
(9, 'HouseNumber', 'TEXT', 0, None, 0)
(10, 'Box', 'TEXT', 0, None, 0)
(11, 'ExtraAddressInfo', 'TEXT', 0, None, 0)
(12, 'DateStrikingOff', 'TEXT', 0, None, 0)


In [ ]:
cursor.execute("""
SELECT enterprise.EnterpriseNumber, address.EntityNumber
FROM enterprise
JOIN address ON enterprise.EnterpriseNumber = address.EntityNumber
LIMIT 10;
""")

for row in cursor.fetchall():
    print(row)

('0200.065.765', '0200.065.765')
('0200.068.636', '0200.068.636')
('0200.171.970', '0200.171.970')
('0200.245.711', '0200.245.711')
('0200.305.493', '0200.305.493')
('0200.362.210', '0200.362.210')
('0200.362.408', '0200.362.408')
('0200.420.410', '0200.420.410')
('0200.420.608', '0200.420.608')
('0200.448.421', '0200.448.421')


In [ ]:
cursor.execute("SELECT COUNT(DISTINCT Zipcode) FROM address;")
print("Zipcodes vides :", cursor.fetchall())


Zipcodes vides : [(38926,)]


In [ ]:
cursor.execute("SELECT COUNT(*) FROM main;")
count = cursor.fetchone()[0]
print(f"Numbre of rows in main : {count}")

Numbre of rows in main : 20


In [ ]:
cursor.execute("SELECT * FROM main LIMIT 5;")
for row in cursor.fetchall():
    print(row)

('0403.465.857', 'Scandinavian Airlines System', 62100.0, 'Unknown', None, 'MAIN', '01-01-1968', 6)
('0403.465.857', 'Scandinavian Airlines System', 51100.0, 'Unknown', None, 'MAIN', '01-01-1968', 6)
('0403.465.857', 'Scandinavian Airlines System', 62100.0, 'Unknown', None, 'MAIN', '01-01-1968', 1)
('0403.465.857', 'Scandinavian Airlines System', 51100.0, 'Unknown', None, 'MAIN', '01-01-1968', 1)
('0403.465.857', 'Scandinavian Airlines System', 51100.0, 'Unknown', None, 'MAIN', '01-01-1968', 6)


In [ ]:
# Close connexion
conn.commit()
conn.close()